<a href="https://colab.research.google.com/github/tjtyler/MachLearn_DecisionTree_ID3/blob/main/lab_3_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Lab

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import math 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [6]:
#--------NODE CLASS-------------
class Node():

  def __init__(self, X, y, feat_indx=None, parent=None, is_leaf=False, is_root=False):
    """
    Args:
        parent_entropy [float]: the entropy of the parent node
        data_array [2D numpy array]: this is an array of the input and target data that only that has been sliced to only include elements filtered by its' parents
        attr_count [int]: this is the value from the count list that corresponds to the attribute being considered (the number of attributes in the category remaining)
    """
    self.X = X # input features
    self.y = y # output targets
    self.parent = parent
    self.is_root = is_root
    self.is_leaf_node = is_leaf
    self.feat_indx = feat_indx
    self.children = []
    self.majority_class = None
    self.entropy = None
    self.proportions = []
    self.counts = []
    self.children_Xs = []
    self.children_ys = []

  def calc_entropy(self):
    self.calc_majority_class()
    xy_concat = np.concatenate((self.X,self.y), axis=1)
    self.counts = self.calc_counts(xy_concat)
    wts = self.calc_wts(xy_concat) # wts = array([float,...,float])
    props = self.calc_props(xy_concat)
    self.entropy = self.calc_WTavrg_entropy(wts, props)
    if self.entropy == 0:
      self.is_leaf_node = True

  def calc_majority_class(self):
    counts = np.bincount(self.y.flatten())
    self.majority_class = np.argmax(counts)

  def calc_counts(self, data):
    counts = []
    for i in range(data.shape[1]):
      counts.append(np.bincount(data[:,i]).shape[0])
    return counts

  
  def calc_wts(self,xy_concat):
    counts = np.bincount(xy_concat[:,self.feat_indx]) # counts = array([int,...,int]): these are counts of each attr 
    wts = counts / np.sum(counts) # wts = array([float,...,float]): these are proportions (counts_per_attr / total_count)
    return wts

  def calc_props(self,xy_concat):
    """
    xy_concat: the X (input) and y (targets) concatenated
    val: the attribute being considered (the column # in the input array, X)
    """
    props = []
    for j in range(self.counts[self.feat_indx]):
      arr = xy_concat[xy_concat[:,self.feat_indx] == j] # this will make a sub-array from xy_concat that is just the rows that have an element in the 'self.feat_indx' column that is equal to 'val'
      self.children_Xs.append(arr[:,:-1]) 
      self.children_ys.append(arr[:,-1:])
      counts = np.bincount(arr[:,-1]) # counts (array([int,...,int])) = the number of each class in our sub-array 
      prop = counts / np.sum(counts) # props (array([float,...,float])) = the proportions of the 
      props.append(prop)
    return props
  
  def calc_WTavrg_entropy(self,wts, props_list):
    """
    wts (numpy array): array([float,...,float])
    props_list (list of numpy arrays): [array([float,...,float]),...,array([float,...,float])]
    """
    avrg_ent = 0
    for i in range(wts.shape[0]):
      ent = 0
      for j in range(props_list[i].shape[0]):
        if props_list[i][j] != 0:
          ent += -props_list[i][j]*math.log2(props_list[i][j])
      avrg_ent += wts[i]*ent
    return avrg_ent
  
  def get_entropy(self):
    return self.entropy
  
  def get_majority_class(self):
    return self.majority_class

  def get_is_leaf(self):
    return self.is_leaf_node

In [ ]:
import math 
def calc_entropy(proportions_tup):
  num_elements = len(proportions_tup)
  entropy = 0
  for i in range(num_elements):
    if proportions_tup[i] != 0:
      entropy += -proportions_tup[i]*math.log2(proportions_tup[i])
  return entropy

def calc_avrg_entropy(attr_props, sub_props):
  """
  attr_props is the overall proportions of the class
  sub_props is a 2D python list of np.arrays that contain the proportions of each attr within a class

  return a list of average entropies for class
  """
  avrg_entropies = []
  for i in range(len(attr_props)-1):
    entropy = 0
    for j in range(attr_props[i].shape[0]):
      entropy += attr_props[i][j]*calc_entropy(sub_props[i][j])
    avrg_entropies.append(entropy)
  return avrg_entropies
  

In [ ]:
HW_data = np.array([['Y', 'Thin', 'N', 'Great'],
                    ['N','Deep', 'N', 'Bad'],
                    ['N', 'Stuffed', 'Y','Good'],
                    ['Y', 'Stuffed','Y','Great'],
                    ['Y','Deep','N','Good'],
                    ['Y','Deep','Y','Great'],
                    ['N','Thin','Y','Good'],
                    ['Y','Deep','N','Good'],
                    ['N','Thin','N','Bad']])
# N = 0; Y = 1; 
# Deep = 0; Stuffed = 1; Thin = 2
# Bad = 0; Good = 1; Great = 2

HW_data[HW_data == 'Y'] = 1
HW_data[HW_data == 'N'] = 0
HW_data[HW_data == 'Deep'] = 0
HW_data[HW_data == 'Stuffed'] = 1
HW_data[HW_data == 'Thin'] = 2
HW_data[HW_data == 'Bad'] = 0
HW_data[HW_data == 'Good'] = 1
HW_data[HW_data == 'Great'] = 2
HW_data = HW_data.astype(int)
print(HW_data)

counts = []
for i in range(HW_data.shape[1]):
  counts.append(np.bincount(HW_data[:,i]).shape[0])
print('counts: \n', counts)


proportions_list = []
for col in range(HW_data.shape[1]):
  prop = np.bincount(HW_data[:,col])
  prop = prop / np.sum(prop)
  proportions_list.append(prop)
print(proportions_list)

parent_node_entropy = calc_entropy(proportions_list[-1])

# children_entropies = []
# for i in range(len(proportions_list)):
#   children_entropies.append(calc_entropy(proportions_list[i]))
# print('children_entropies', children_entropies)
sub_attr_proportions = []
for i in range(len(proportions_list)-1):
  attr_props = []
  for j in range(counts[i]):
    data_arr = HW_data[HW_data[:,i] == j] 
    print('data:\n',data_arr)
    prop = np.bincount(data_arr[:,-1])
    prop = prop/np.sum(prop)
    attr_props.append(prop)
  sub_attr_proportions.append(attr_props)
print('sub_attr_proportions\n', sub_attr_proportions)

for i in range(len(sub_attr_proportions)):
  for j in range(len(sub_attr_proportions[i])):
    for prop in sub_attr_proportions[i][j]:
      print('prop = ', prop)

children_avrg_entropies = calc_avrg_entropy(proportions_list, sub_attr_proportions)
print('children_avrg_entropies:\n',children_avrg_entropies)

information_gained = []
for i in range(len(children_avrg_entropies)):
  info_gained = parent_node_entropy - children_avrg_entropies[i]
  information_gained.append(info_gained)

print('information_gained:\n', information_gained)

node_to_split_on = information_gained.index(max(information_gained))
print('node_to_split_on ', node_to_split_on)
# zero_meat = HW_data[HW_data[:,0] == 0] 
# print('no meat data:\n',zero_meat)
# prop = np.bincount(zero_meat[:,-1])
# print(prop/np.sum(prop))




[[1 2 0 2]
 [0 0 0 0]
 [0 1 1 1]
 [1 1 1 2]
 [1 0 0 1]
 [1 0 1 2]
 [0 2 1 1]
 [1 0 0 1]
 [0 2 0 0]]
counts: 
 [2, 3, 2, 3]
[array([0.44444444, 0.55555556]), array([0.44444444, 0.22222222, 0.33333333]), array([0.55555556, 0.44444444]), array([0.22222222, 0.44444444, 0.33333333])]
data:
 [[0 0 0 0]
 [0 1 1 1]
 [0 2 1 1]
 [0 2 0 0]]
data:
 [[1 2 0 2]
 [1 1 1 2]
 [1 0 0 1]
 [1 0 1 2]
 [1 0 0 1]]
data:
 [[0 0 0 0]
 [1 0 0 1]
 [1 0 1 2]
 [1 0 0 1]]
data:
 [[0 1 1 1]
 [1 1 1 2]]
data:
 [[1 2 0 2]
 [0 2 1 1]
 [0 2 0 0]]
data:
 [[1 2 0 2]
 [0 0 0 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 2 0 0]]
data:
 [[0 1 1 1]
 [1 1 1 2]
 [1 0 1 2]
 [0 2 1 1]]
sub_attr_proportions
 [[array([0.5, 0.5]), array([0. , 0.4, 0.6])], [array([0.25, 0.5 , 0.25]), array([0. , 0.5, 0.5]), array([0.33333333, 0.33333333, 0.33333333])], [array([0.4, 0.4, 0.2]), array([0. , 0.5, 0.5])]]
prop =  0.5
prop =  0.5
prop =  0.0
prop =  0.4
prop =  0.6
prop =  0.25
prop =  0.5
prop =  0.25
prop =  0.0
prop =  0.5
prop =  0.5
prop =  0.3333333

In [ ]:
HW_data = np.array([['Y', 'Thin', 'N', 'Great'],
                    ['N','Deep', 'N', 'Bad'],
                    ['N', 'Stuffed', 'Y','Good'],
                    ['Y', 'Stuffed','Y','Great'],
                    ['Y','Deep','N','Good'],
                    ['Y','Deep','Y','Great'],
                    ['N','Thin','Y','Good'],
                    ['Y','Deep','N','Good'],
                    ['N','Thin','N','Bad']])
# N = 0; Y = 1; 
# Deep = 0; Stuffed = 1; Thin = 2
# Bad = 0; Good = 1; Great = 2

HW_data[HW_data == 'Y'] = 1
HW_data[HW_data == 'N'] = 0
HW_data[HW_data == 'Deep'] = 0
HW_data[HW_data == 'Stuffed'] = 1
HW_data[HW_data == 'Thin'] = 2
HW_data[HW_data == 'Bad'] = 0
HW_data[HW_data == 'Good'] = 1
HW_data[HW_data == 'Great'] = 2
HW_data = HW_data.astype(int)
X = HW_data[:,:-1]
y = HW_data[:,-1:]

nodes = []
nodes_entropies = []
for i in range(X.shape[1]):
  node = Node(X,y,i)
  node.calc_entropy()
  print('node entropy ', node.get_entropy())
  nodes_entropies.append(node.get_entropy())
  nodes.append(node)
next_node_indx = nodes_entropies.index(min(nodes_entropies))
root_node = nodes[next_node_indx]

print(root_node.get_majority_class())
print(root_node.get_entropy())


node entropy  0.9838614413637048
node entropy  1.4172097224626075
node entropy  1.2899600527152013
1
0.9838614413637048


In [5]:
class Node():
  def __init__(self,X,y,par_node=None, par_entrop=None, node_feat=None, leaf=False,split_indx=None, depth=0):
    self.X=X
    self.y=y
    self.entropy = self.entropy()
    self.parent_entropy = par_entrop
    self.parent_node = par_node
    self.children = []
    self.leaf = leaf
    self.depth = depth
    self.split_indx = split_indx #index of attrib to split on
    self.maj_class = None
    self.info_gained = None
    self.counts = self.calc_counts()
    self.node_feat = node_feat

  def entropy(self):
    quants = np.bincount(self.y.flatten())
    props = quants / np.sum(quants)
    entrop = 0
    for i in range(len(quants)):
      if props[i] != 0:
        entrop += -props[i]*math.log2(props[i])
    return entrop
  
  def calc_counts(self):
    xy_concat = np.concatenate((self.X, self.y.reshape(self.y.shape[0],1)), axis=1)
    cnts = []
    for i in range(xy_concat.shape[1]):
      cnts.append(np.bincount(xy_concat[:,i]).shape[0])
    return cnts


  def set_split_indx(self, split_indx):
    self.split_indx = split_indx
  
  def set_children(self, children):
    self.children = children

  def set_is_leaf(self,leaf):
    self.leaf = leaf
  
  def set_majority_cls(self, cls):
    self.maj_class = cls

  def set_info_gained(self,info_gained):
    self.info_gained = info_gained


In [12]:
class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self):
        """ Initialize class with chosen hyperparameters.
        Args:
        Optional Args (Args we think will make your life easier):
            counts: A list of Ints that tell you how many types of each feature there are
        Example:
            DT  = DTClassifier()
            or
            DT = DTClassifier(count = [2,3,2,2])
            Dataset = 
            [[0,1,0,0],
            [1,2,1,1],
            [0,1,1,0],
            [1,2,0,1],
            [0,0,1,1]]

        """
        self.information_gained = []

    def fit(self, X, y):
        """ Fit the data; Make the Decision tree

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 1D numpy array with the training targets

        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        # self.root = Node()
        # self.make_tree(self.root)
        self.root = Node(X=X,y=y,par_node=None, par_entrop=None, node_feat=None, leaf=False,split_indx=None,depth=0)
        self.make_tree(self.root)
        return self

    def make_tree(self,node):
      # IF BASE CASE
      # Base Case:
        # if it's supposed to be a leaf node, make it a leaf node and set its majority class
        # return
      if node.entropy == 0:
        node.set_is_leaf(True)
        cls = node.y[0]
        node.set_majority_cls(cls)
        node.set_info_gained(node.entropy)
        return

      # IF THE NODE IS NOT A LEAF NODE THEN: 
        # DO STUFF
        # Finding the best feature to split on
        # Make the split (create all the children nodes) 
      else:
        # FIND THE BEST FEATURE TO SPLIT ON
        pos_children = [] # possible features to become children [[]]
        avrg_children_ents = []

        for ft_indx in range(node.X.shape[1]): # these are the columns of the X array
          feat_cnts = np.bincount(node.X[:,ft_indx]) # this is a 1D np array that contains at each indx of the np.arr how many numbers are in in the column
          feat_props = feat_cnts / np.sum(feat_cnts) # this is the proportions of each attribute of feat[ft_indx]
          feat_children = [] # this will be a list of the children nodes of feat[ft_indx]
          avrg_ent = 0 # average entropy of a feature's children
          # LOOP OVER ALL OF THE ATTRIBUTES OF THE FEATURE (if counts = [3,4,3] and the feature is 0, then we loop from 0 to 2)
          for unq_val in range(node.counts[ft_indx]):
            xy_concat = np.concatenate((node.X, node.y.reshape(node.y.shape[0],1)), axis=1) # concatenate the X and y from the node that was passed in
            Xy_child = xy_concat[xy_concat[:,ft_indx] == unq_val] # create the sub array for this child where the attribs of the feat = unq_val
            child_X = Xy_child[:,:-1] # split Xy_child to get the X for the child
            child_y = Xy_child[:,-1] # split Xy_child to get teh y for the child
            child = Node(X=child_X,y=child_y,par_node=node, par_entrop=node.parent_entropy, node_feat=unq_val, depth=node.depth+1) # create the child node and increment its depth
            avrg_ent += child.entropy*feat_props[unq_val] # add this child's entropy to the avrg entropy for this feature
            feat_children.append(child) # add child to this features list of children
          pos_children.append(feat_children) # add the list of children of the feature just condidered to the list of possible children
          avrg_children_ents.append(avrg_ent) # add the avrg entropy of the children of the feature just considered to the lsit of average entropies

        split_feat_indx = avrg_children_ents.index(min(avrg_children_ents)) # feature to split on wlll be the feature with the lowest avrg entropy
        children_nodes = pos_children[split_feat_indx] # the children nodes of the feature to split on become the children nodes to keep
        
        node.set_split_indx(split_feat_indx) # set the splt_indx of the original node passed in
        node.set_children(children_nodes) # assign the children to the original node passed in
        # print('node.entropy: ', node.entropy)
        # print('avrg_children_ents[split_feat_indx]: ', avrg_children_ents[split_feat_indx])
        node.set_info_gained(node.entropy - avrg_children_ents[split_feat_indx])
        self.information_gained.append(node.info_gained)
        
        # RECURSE ON CHILDREN
        for child in children_nodes:
          self.make_tree(child)

    def print_tree(self):
      self.print_tree_helper(self.root)

    def print_tree_helper(self, node):
    
      # either print for a split node or print for a leaf node
      if node.leaf:
        print('\t'*node.depth + f'prediction = {node.maj_class}')
      # call print_tree on each child node
      # ROOT NODE
      else:
        for child in node.children:
          print('\t'*node.depth + f'feature {node.split_indx} = ' + f'{child.node_feat}')
          self.print_tree_helper(child)
          

    def predict(self, X):
        """ Predict all classes for a dataset X

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets

        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        predictions = []
        # LOOP OVER THE INSTANCES OF THE DATA
        for i in range(X.shape[0]):
          inst = X[i]       
          cur_node = self.root
          
          while not cur_node.leaf:
            child_indx = inst[cur_node.split_indx]
            cur_node = cur_node.children[child_indx]
          predictions.append(cur_node.maj_class)
        return predictions


    def score(self, X, y, shuffle=True):
        """ Return accuracy(Classification Acc) of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 1D numpy array of the targets 
        """
        if shuffle:
          X,y = self._shuffle_data(X,y)
        predictions = self.predict(X)
        correct = 0
        total = y.shape[0]
        for i in range(0, y.shape[0]):
          if predictions[i] == y[i][0]:
            correct += 1
        return correct/total

    def _shuffle_data(self, X, y):
      """ 
          Shuffle the data! This _ prefix suggests that this method should 
          only be called internally.
          It might be easier to concatenate X & y and shuffle a single 2D 
          array, rather than shuffling X and y exactly the same way, 
          independently.
      """
      single_arr = np.concatenate((X,y), axis=1) # concatenate X and y into a single array
      np.random.shuffle(single_arr) # shuffle the rows of the concatenated X-y array
      cutoff = single_arr.shape[1] - 1 # the point to split the X and y arrays after shuffling
      X = single_arr[:,:cutoff] # the shuffled X array
      y = single_arr[:,cutoff:] # the shuffled y array
      return X,y

    def get_information_gained(self):
      return self.information_gained

In [89]:
# DEBUG HOMEWORK PIZZA PROBLEM

HW_data = np.array([['Y', 'Thin', 'N', 'Great'],
                    ['N','Deep', 'N', 'Bad'],
                    ['N', 'Stuffed', 'Y','Good'],
                    ['Y', 'Stuffed','Y','Great'],
                    ['Y','Deep','N','Good'],
                    ['Y','Deep','Y','Great'],
                    ['N','Thin','Y','Good'],
                    ['Y','Deep','N','Good'],
                    ['N','Thin','N','Bad']])
# N = 0; Y = 1; 
# Deep = 0; Stuffed = 1; Thin = 2
# Bad = 0; Good = 1; Great = 2

HW_data[HW_data == 'Y'] = 1
HW_data[HW_data == 'N'] = 0
HW_data[HW_data == 'Deep'] = 0
HW_data[HW_data == 'Stuffed'] = 1
HW_data[HW_data == 'Thin'] = 2
HW_data[HW_data == 'Bad'] = 0
HW_data[HW_data == 'Good'] = 1
HW_data[HW_data == 'Great'] = 2
HW_data = HW_data.astype(int)
X = HW_data[:,:-1]
y = HW_data[:,-1:]

decision_tree = DTClassifier()
decision_tree.fit(X,y)
score = decision_tree.score(X=X,y=y)
print('score ', score)
decision_tree.print_tree()

score  1.0
feature 0 = 0
	feature 2 = 0
		prediction = 0
	feature 2 = 1
		prediction = 1
feature 0 = 1
	feature 1 = 0
		feature 2 = 0
			prediction = 1
		feature 2 = 1
			prediction = 2
	feature 1 = 1
		prediction = 2
	feature 1 = 2
		prediction = 2


## 1.1 Debug

Debug your model by training on the lenses dataset: [Debug Dataset (lenses.arff)](https://byu.instructure.com/courses/14142/files?preview=4622251)

Test your model on the lenses test set: [Debug Test Dataset (lenses_test.arff)](https://byu.instructure.com/courses/14142/files?preview=4622254)

Parameters:
(optional) counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions (pred_lenses.csv)](https://byu.instructure.com/courses/14142/files?preview=4622260)

*NOTE: The [Lenses Prediction (pred_lenses.csv)](https://byu.instructure.com/courses/14142/files?preview=4622260) uses the following encoding: soft=2, hard=0, none=1. If your encoding is different, then your output will be different, but not necessarily incorrect.*

Split Information Gains (These do not need to be in this exact order):

[0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits should look like, and the corresponding child node predictions:

Decision Tree:
<pre>
tear_prod_rate = normal:
    astigmatism = no:
        age = pre_presbyopic:
            prediction: soft
        age = presbyopic:
            spectacle_prescrip = hypermetrope:
                prediction: soft
            spectacle_prescrip = myope:
                prediction: none
        age = young:
            prediction: soft
    astigmatism = yes:
        spectacle_prescrip = hypermetrope:
            age = pre_presbyopic:
                prediction: none
            age = presbyopic:
                prediction: none
            age = young:
                prediction: hard
        spectacle_prescrip = myope:
            prediction: hard
tear_prod_rate = reduced:
    prediction: none
</pre>

In [13]:
from scipy.io.arff import loadarff 
import pandas as pd

# Load debug training data
raw_data = loadarff('/content/drive/MyDrive/School/CS_472_MachLearning/labs/lab3_decisionTree/data/lenses.arff')
df_data = pd.DataFrame(raw_data[0])

np_arr = df_data.to_numpy() #cast dataframe to numpy array
np_arr[np_arr == b'young'] = 0 
np_arr[np_arr == b'pre_presbyopic'] = 1 
np_arr[np_arr == b'presbyopic'] = 2 

np_arr[np_arr == b'myope'] = 0 
np_arr[np_arr == b'hypermetrope'] = 1 

np_arr[np_arr == b'no'] = 0
np_arr[np_arr == b'yes'] = 1

np_arr[np_arr == b'reduced'] = 0
np_arr[np_arr == b'normal'] = 1

np_arr[np_arr == b'soft'] = 0
np_arr[np_arr == b'hard'] = 1
np_arr[np_arr == b'none'] = 2

np_arr = np_arr.astype(int)

training_data = np_arr.copy()

X_train = training_data[:,:-1]
y_train = training_data[:,-1:]

# Train Decision Tree
decision_tree = DTClassifier()
decision_tree.fit(X_train,y_train)


# Load debug test data
raw_data = loadarff('/content/drive/MyDrive/School/CS_472_MachLearning/labs/lab3_decisionTree/data/lenses_test.arff')
df_data = pd.DataFrame(raw_data[0])

np_arr = df_data.to_numpy() #cast dataframe to numpy array
np_arr[np_arr == b'young'] = 0 
np_arr[np_arr == b'pre_presbyopic'] = 1 
np_arr[np_arr == b'presbyopic'] = 2 

np_arr[np_arr == b'myope'] = 0 
np_arr[np_arr == b'hypermetrope'] = 1 

np_arr[np_arr == b'no'] = 0
np_arr[np_arr == b'yes'] = 1

np_arr[np_arr == b'reduced'] = 0
np_arr[np_arr == b'normal'] = 1

np_arr[np_arr == b'soft'] = 0
np_arr[np_arr == b'hard'] = 1
np_arr[np_arr == b'none'] = 2

np_arr = np_arr.astype(int)

test_data = np_arr.copy()

X_test = test_data[:,:-1]
y_test = test_data[:,-1:]

# Predict and compute model accuracy
score = decision_tree.score(X=X_test,y=y_test)
print('Accuracy: ', score)

# Print the information gain of every split you make.
print('information gained: ', decision_tree.get_information_gained())


Accuracy:  0.3333333333333333
information gained:  [0.5487949406953982, 0.7704260414863778, 0.3166890883150208, 1.0, 0.4591479170272448, 0.9182958340544896]


In [ ]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset (zoo.arff)](https://byu.instructure.com/courses/14142/files?preview=4622270)

Test your model on this dataset: [Evaluation Test Dataset (zoo_test.arff)](https://byu.instructure.com/courses/14142/files?preview=4622274)

Parameters:
(optional) counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)

---
Print out your accuracy on the evaluation test dataset.

Print out the information gain of every split you make.

In [ ]:
# Load evaluation training data


# Train Decision Tree


# Load evaluation test data


# Print out the information gain for every split you make



## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criterion, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [ ]:
# Write a function that implements 10-fold cross validation

##  2.2 Cars Dataset
- Use this [Cars Dataset (cars.arff)](hhttps://byu.instructure.com/courses/14142/files?preview=4622293)
- Make a table for your k-fold cross validation accuracies

*If you are having trouble using scipy's loadarff function (scipy.io.arff.loadarff), try:*

*pip install arff &nbsp;&nbsp;&nbsp;&nbsp;          # Install arff library*

*import arff as arf*                   

*cars = list(arf.load('cars.arff'))   &nbsp;&nbsp;&nbsp;&nbsp;# Load your downloaded dataset (!curl, etc.)*

*df = pd.DataFrame(cars)*  

*There may be additional cleaning needed*

In [ ]:
# Use 10-fold CV on Cars Dataset

# Report Training and Test Classification Accuracies

# Report Average Test Accuracy

## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values (voting_with_missing.arff)](https://byu.instructure.com/courses/14142/files?preview=4622298)
- Note that you will need to support unknown attributes in the voting data set. 

In [ ]:
# Used 10-fold CV on Voting Dataset

# Report Training and Test Classification Accuracies

# Report Average Test Accuracy

## 2.4 Discuss Your Results

- Summarize your results from both datasets, and discuss what you observed. 
- A fully expanded tree will often get 100% accuracy on the training set. Why does this happen and in what cases might it not?  

Discuss your results

## 3. (15%) For each of the two problems above, summarize in English what the decision tree has learned (i.e., look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

## 3.1 Discuss what the decision tree induced on the cars dataset has learned

Discussion Here

## 3.2 Discuss what the decision tree induced on the voting dataset has learned

Discussion Here

## 3.3 How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

Discuss how you handled unknown attributes

## 4.1 (10%) Use Scikit Learn's decision tree on the voting dataset and compare your results. Try different parameters and report what parameters perform best on the test set. 

### 4.1.1 sklearn on Voting Dataset
- Use this [Voting Dataset with missing values (voting_with_missing.arff)](https://byu.instructure.com/courses/14142/files?preview=4622298)

In [ ]:
# Use sklearn's Decision Tree to learn the voting dataset

# Explore different parameters

# Report results

Discuss results & compare to your method's results

## 4.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the sklearn decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

In [ ]:
# Use sklearn's Decision Tree on a new dataset

# Experiment with different hyper-parameters

## 5. (5%) Visualize sklearn's decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first few levels (e.g., top 5).

In [ ]:
# Include decision tree visualization here

# Discuss what the model has learned

## 6. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).